In [31]:
using Pkg
Pkg.add("DifferentialEquations")
Pkg.add("Interact")

 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed CSSUtil ────── v0.1.0
 Installed Knockout ───── v0.2.2
 Installed Interact ───── v0.10.2
 Installed InteractBase ─ v0.10.1
  Updating `~/.julia/environments/v1.1/Project.toml`
  [c601a237] + Interact v0.10.2
  Updating `~/.julia/environments/v1.1/Manifest.toml`
  [70588ee8] + CSSUtil v0.1.0
  [c601a237] + Interact v0.10.2
  [d3863d7c] + InteractBase v0.10.1
  [bcebb21b] + Knockout v0.2.2
  Building Knockout ────→ `~/.julia/packages/Knockout/JE2Yq/deps/build.log`
  Building InteractBase → `~/.julia/packages/InteractBase/R8JgH/deps/build.log`
  Building Interact ────→ `~/.julia/packages/Interact/0klKX/deps/build.log`


In [4]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
# https://calculuswithjulia.github.io/ODEs/euler.html - Euler in Julia
# http://docs.juliadiffeq.org/latest/tutorials/ode_example.html#Ordinary-Differential-Equations-1 - ODE in Julia
# http://pages.physics.cornell.edu/~sethna/teaching/sss/galileo/galil02.htm


#damped pendulum
#x'' = -bx' -g/l sin(x)
#theta''(t) + b*theta'(t) + g/l*sin(theta(t)) = 0
#theta''(t) = -b*theta'(t) - g/l*sin(theta(t))

#eq system:
#theta'(t) = omega(t)  -- speed
#omega'(t) = -b*omega(t) - g/l*sin(theta(t))


using DifferentialEquations
using Interact
using Plots

function create_pi_ticks(x0, step, xn)
    xspan = x0:step:xn
    n = length(xspan)
    x = []
    y = String[]
    for i = 1:n
        push!(x,xspan[i]*pi)
        push!(y,string(xspan[i],"\\pi"))
    end
    return (x,y)
end

function euler2(x0, xn, theta0, omega0, n, g, l, b)
    xs, us, vs = zeros(n+1), zeros(n+1), zeros(n+1)
    xs[1], us[1], vs[1] = x0, theta0, omega0
    h = (xn - x0)/n
    for i = 1:n
        xs[i+1] = xs[i] + h
        us[i+1] = us[i] + h * vs[i]
        vs[i+1] = vs[i] + h * (-b*vs[i] + (-g/l) * sin(us[i]))
    end
    return (xs,us,vs)
end

@manipulate for 
    method = ["euler", "julia"],
    plot_type  = ["u1, u2", "2D"],
    t in slider(0:0.1:30, value=5, label="time"),
    g in slider(1:0.1:20, value=9.8, label="gravity"),
    l in slider(0.01:0.01:2, value = 1, label="length"),
    theta0 in slider(-π:π/32:π, value = pi*0.5, label="theta0 (start angle)"),
    omega0 in slider(-10:0.1:10, value = 0, label="omega0 (start speed)"),
    b in slider(0:0.01:2, value = 0, label="b"),
    n in slider(200:1:1000, value = 360, label="n (euler)")

    if method == "julia"
        function equation(du,u,p,t)
            #u1 - theta
            #u2 - omega
            du[1] = u[2]
            du[2] = -b*u[2] -g/l * sin(u[1])
        end
        
        u0 = [theta0, omega0]
        tspan = (0.0,t)
        prob = ODEProblem(equation,u0,tspan)
        sol = solve(prob)
        if plot_type == "u1, u2"
            plot(sol, label=["position", "speed"])
        elseif plot_type == "2D"
            plot(sol, vars=(1,2))
        end
    elseif method == "euler"
        xs, theta, omega = euler2(0, t, theta0, omega0, n, g, l, b)
        if plot_type =="u1, u2"
            plot(xs, theta, label="position") 
            plot!(xs, omega, label="speed") 
        elseif plot_type=="2D"
            plot(theta,omega) 
        end
    end
    if plot_type =="u1, u2"
        plot!(yticks = create_pi_ticks(-5, 0.25, 5), xlabel="time", ylabel="position")
    elseif plot_type=="2D"
        plot!(xticks = create_pi_ticks(-5, 0.25, 5), xlabel="position", ylabel="speed")
    end
    
end



Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope("knockout-component-4d3eb3d1-8cdf-49c2-bdc5-2efd16d11b6f", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["method"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :span), Any[Node{DOM}(DOM(:html, :label), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text : key")), 0)], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget button ' : true}")), 1)], Dict{Symbol,Any}(:attributes=>("data-bind"=>"foreach : options_js"),:className=>"buttons has-addons is-centered"), 2)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 3), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[], Dict{Symbol,Any}(), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 9), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("options_js"=>(Observable{Any} with 1 listeners. Value:
OrderedDict[OrderedDict{String,Any}("key"=>"euler","val"=>1,"id"=>"id5NtYL4H8"), OrderedDict{String,Any}("key"=>"julia","val"=>2,"id"=>"idPUl06LCo")], nothing),"index"=>(Observable{Any} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/home/jakubdybczak/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/jakubdybczak/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/jakubdybczak/.julia/packages/InteractBase/R8JgH/src/../assets/all.js"), Asset("css", nothing, "/home/jakubdybczak/.julia/packages/InteractBase/R8JgH/src/../assets/style.css"), Asset("css", nothing, "/home/jakubdybczak/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("options_js"=>Any[JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"options_js\":WebIO.getval({\"name\":\"options_js\",\"scope\":\"knockout-component-4d3eb3d1-8cdf-49c2-bdc5-2efd16d11b6f\",\"id\":\"ob_113\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"knockout-component-4d3eb3d1-8cdf-49c2-bdc5-2efd16d11b6f\",\"id\":\"ob_112\",\"type\":\"observable\"})};\n    var self = t